In [1]:
from os import system
import os
import pandas as pd
import numpy as np
import time
import copy
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import graphviz
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import lr_scheduler
from sklearn.metrics import accuracy_score

# Data Preprocessing (No need to use in case of pickle load)

In [2]:
data = pd.read_csv('flight.csv')
# data2 = pd.read_csv('dataset/flight_littleone.csv')
# data3 = pd.read_csv('dataset/flight_mustafa.csv')
# data4 = pd.read_csv('dataset/flight_ubuntu.csv')
# frames = [data1, data2, data3, data4]
# result = pd.concat(frames)
N_B1 = np.sum(data["controller_ID"] == "Backstepping_1")
N_B2 = np.sum(data["controller_ID"] == "Backstepping_2")
N_B3 = np.sum(data["controller_ID"] == "Backstepping_3")
N_B4 = np.sum(data["controller_ID"] == "Backstepping_4")

print("Dataset size: ", data.shape)
print("Backstepping_1 (the most agile): %", N_B1/(N_B1+N_B2+N_B3+N_B4)*100)
print("Backstepping_2 (agile): %", N_B2/(N_B1+N_B2+N_B3+N_B4)*100)
print("Backstepping_3 (smooth): %", N_B3/(N_B1+N_B2+N_B3+N_B4)*100)
print("Backstepping_4 (the smoothest): %", N_B4/(N_B1+N_B2+N_B3+N_B4)*100)
data[0:10]

Dataset size:  (635580, 38)
Backstepping_1 (the most agile): % 48.107712640422925
Backstepping_2 (agile): % 1.7302306554643
Backstepping_3 (smooth): % 6.893388715818623
Backstepping_4 (the smoothest): % 43.268667988294155


x0        y0        z0    x_dot0    y_dot0    z_dot0      phi0  \
0 -1.900659  0.280235  1.696418 -4.579493  2.070832 -3.238523 -0.145760   
1 -1.905235  0.282304  1.693181 -4.572434  2.067247 -3.236889 -0.145536   
2 -1.909804  0.284370  1.689944 -4.565362  2.063656 -3.235251 -0.145261   
3 -1.914366  0.286432  1.686710 -4.558282  2.060063 -3.233609 -0.144858   
4 -1.918920  0.288490  1.683477 -4.551201  2.056470 -3.231963 -0.144328   
5 -1.923468  0.290545  1.680246 -4.544126  2.052881 -3.230312 -0.143671   
6 -1.928009  0.292596  1.677017 -4.537063  2.049301 -3.228656 -0.142887   
7 -1.932542  0.294643  1.673789 -4.530018  2.045731 -3.226997 -0.141979   
8 -1.937069  0.296687  1.670563 -4.522998  2.042176 -3.225333 -0.140947   
9 -1.941588  0.298728  1.667338 -4.516008  2.038637 -3.223666 -0.139796   

     theta0      yaw0  phi_dot0  ...        yp        zp    x_dotp    y_dotp  \
0  0.589219 -0.728182  0.235309  ...  0.280235  1.696418 -4.579493  2.070832   
1  0.589948 -0.727442  0.212167  ...  0.280235  1.696418 -4.579493  2.070832   
2  0.590609 -0.726654  0.338972  ...  0.282305  1.693181 -4.577105  2.068945   
3  0.590787 -0.725777  0.466631  ...  0.284373  1.689944 -4.574717  2.067061   
4  0.590509 -0.724811  0.594331  ...  0.286439  1.686710 -4.572329  2.065179   
5  0.589799 -0.723761  0.721346  ...  0.288504  1.683477 -4.569941  2.063299   
6  0.588683 -0.722627  0.847045  ...  0.290566  1.680246 -4.567553  2.061421   
7  0.587185 -0.721412  0.970884  ...  0.292626  1.677017 -4.565166  2.059546   
8  0.585328 -0.720119  1.092399  ...  0.294685  1.673789 -4.562778  2.057673   
9  0.583133 -0.718749  1.211204  ...  0.296742  1.670563 -4.560390  2.055802   

     z_dotp   x_ddotp   y_ddotp   z_ddotp     u_abs_p   controller_ID  
0 -3.238523  2.387939 -1.887509  1.637147    1.000000  Backstepping_4  
1 -3.238523  2.387939 -1.887509  1.637147  209.736982  Backstepping_1  
2 -3.236885  2.387934 -1.885310  1.639824  337.163728  Backstepping_1  
3 -3.235244  2.387929 -1.883113  1.642499  321.673448  Backstepping_1  
4 -3.233600  2.387924 -1.880916  1.645171  306.518919  Backstepping_1  
5 -3.231953  2.387918 -1.878721  1.647841  291.675474  Backstepping_1  
6 -3.230304  2.387911 -1.876527  1.650509  277.184081  Backstepping_1  
7 -3.228652  2.387904 -1.874335  1.653174  263.078594  Backstepping_1  
8 -3.226998  2.387897 -1.872143  1.655837  249.386166  Backstepping_1  
9 -3.225341  2.387889 -1.869953  1.658497  236.127719  Backstepping_1  

[10 rows x 38 columns]

In [8]:
N_Test = 65000
dataset = data.values
controller_labels = {'Backstepping_1': 0, 'Backstepping_2': 1, 'Backstepping_3': 2, 'Backstepping_4': 3}
np.random.shuffle(dataset)
y = np.array([controller_labels[data[-1]] for data in dataset]).reshape(-1,)
X = dataset[:,:-1]

X_test = X[0:N_Test,:]
y_test = y[0:N_Test]

X_train, X_val, y_train, y_val = train_test_split(X[N_Test:,:], y[N_Test:], test_size=0.15, random_state=42)

# Saving the objects:
# with open('dataset.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
#     pickle.dump([X_train, X_val, X_test, y_train, y_val, y_test], f)
    
pickle.dump([X_train, X_val, X_test, y_train, y_val, y_test], open("dataset.pkl","wb"), protocol=2)


scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

print ("X_train size: ",X_train.shape)
print ("X_val size: ",X_val.shape)
print ("X_test size: ",X_test.shape)

X_train size:  (484993, 37)
X_val size:  (85587, 37)
X_test size:  (65000, 37)


# Data Loading and Merging

In [2]:
with open('dataset.pkl', 'rb') as f:  # Python 3: open(..., 'wb')
    X_train, X_val, X_test, y_train, y_val, y_test = pickle.load(f)

with open('dataset_mustafa.pkl', 'rb') as f:  # Python 3: open(..., 'wb')
    X_trainM, X_valM, X_testM, y_trainM, y_valM, y_testM = pickle.load(f)
    
Xtrain = np.concatenate((X_train, X_trainM), axis=0)
Xval = np.concatenate((X_val, X_valM), axis=0)
Xtest = np.concatenate((X_test, X_testM), axis=0)

ytrain = np.concatenate((y_train, y_trainM), axis=0)
yval = np.concatenate((y_val, y_valM), axis=0)
ytest = np.concatenate((y_test, y_testM), axis=0)

scaler = StandardScaler()
scaler.fit(Xtrain)
Xtrain = scaler.transform(Xtrain)
Xval = scaler.transform(Xval)
Xtest = scaler.transform(Xtest)

print ("Xtrain size: ",Xtrain.shape)
print ("Xval size: ",Xval.shape)
print ("Xtest size: ",Xtest.shape)

Xtrain size:  (1326141, 37)
Xval size:  (234026, 37)
Xtest size:  (165000, 37)


# Decision Tree Design

In [57]:
from sklearn.tree._tree import TREE_LEAF

def prune_index(inner_tree, index, threshold):
    if inner_tree.value[index].min() < threshold:
        # turn node into a leaf by "unlinking" its children
        inner_tree.children_left[index] = TREE_LEAF
        inner_tree.children_right[index] = TREE_LEAF
    # if there are shildren, visit them as well
    if inner_tree.children_left[index] != TREE_LEAF:
        prune_index(inner_tree, inner_tree.children_left[index], threshold)
        prune_index(inner_tree, inner_tree.children_right[index], threshold)


feature_names = ['x0', 'y0', 'z0', 'x_dot0','y_dot0','z_dot0', 'phi0','theta0','yaw0', 'phi_dot0','theta_dot0','yaw_dot0', 
                 'xf', 'yf', 'zf', 'x_dotf','y_dotf','z_dotf','x_ddotf','y_ddotf','z_ddotf',
                 'pos_diffx','pos_diffy','pos_diffz','time_rate','t', 'Tf', 
                 'xp', 'yp', 'zp', 'x_dotp','y_dotp','z_dotp','x_ddotp','y_ddotp','z_ddotp', 'u_abs_p']

class_names = ['Backstepping_1', 'Backstepping_2', 'Backstepping_3', 'Backstepping_4']
clf = DecisionTreeClassifier()
clf = clf.fit(Xtrain, ytrain)
# prune_index(clf.tree_, 0, 10)

train_pred = clf.predict(Xtrain)
val_pred = clf.predict(Xval)
test_pred = clf.predict(Xtest)


train_acc = accuracy_score(ytrain, train_pred)
val_acc = accuracy_score(yval, val_pred)
test_acc = accuracy_score(ytest, test_pred)

print ("Decision Tree Train Acc: {0:.3}, Val Acc: {1:.3}, Test Acc: {2:.3}".format(train_acc, val_acc, test_acc))


# forest = RandomForestClassifier(bootstrap=True, n_estimators=100, 
#                                 min_samples_split=12, min_samples_leaf=5, max_features=10, max_depth=80)
# forest = forest.fit(X_train, y_train)

# train_pred = forest.predict(X_train)
# val_pred = forest.predict(X_val)
# test_pred = forest.predict(X_test)


# train_acc = accuracy_score(y_train, train_pred)
# val_acc = accuracy_score(y_val, val_pred)
# test_acc = accuracy_score(y_test, test_pred)

# print ("Random Forest Train Acc: {0:.3}, Val Acc: {1:.3}, Test Acc: {2:.3}".format(train_acc, val_acc, test_acc))

# dot_data = tree.export_graphviz(clf, out_file=None, 
#                       feature_names=feature_names,  
#                       class_names=class_names,  
#                       filled=True, rounded=True,  
#                       special_characters=True)
# graph = graphviz.Source(dot_data) 
# graph

Decision Tree Train Acc: 1.0, Val Acc: 0.996, Test Acc: 0.996


# Neural Network Design

In [3]:
stats_columns = ['Layers', 'Epochs', 'BatchSize', 'LearningRate', 'Optimizer', 'Scheduler', 'TrainAcc', 'ValAcc']
stats_filename = 'params_results.csv'

def write_results(results): 
    df_stats = pd.DataFrame([results], columns=stats_columns)
    df_stats.to_csv(stats_filename, mode='a', index=False,header=not os.path.isfile(stats_filename))

def predict(X, y, model):
    #Validation part
    model.eval()  # Set model to training mode

    inputs, labels = torch.from_numpy(X).to(device), torch.from_numpy(y).to(device).long()

    outputs = model(inputs.float())
    _, preds = torch.max(outputs, 1)

    loss = criterion(outputs, labels)
    
    accuracy = torch.sum(preds == labels.data).item() / float(inputs.size(0))
    
    print ("Test data, Loss: {0:.3}, Accuracy: {1:.4}".format(loss.item(), accuracy))
    

def shuffle_dataset(X, y):
    p = np.random.permutation(len(X))
    return X[p], y[p]

def train_model(X, y, X_val, y_val, model, criterion, optimizer, scheduler, minibatch_size, num_epochs=25):

    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    losses_train = []
    losses_val = []
    accuracy_train = []
    accuracy_val = []
    # path = F"/content/drive/My Drive/best_model.pt"
#     directory = path_name

#     if not os.path.exists(directory):
#         os.makedirs(directory)

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        X_train, y_train = shuffle_dataset(X, y)

        # Each epoch has a training and validation phase
        model.train()  # Set model to training mode
        running_loss = 0.0
        running_corrects = 0
        losses_iter = []
        accuracy_iter = []

        # Iterate over data.
        for i in range(0, X_train.shape[0], minibatch_size):
            # Get pair of (X, y) of the current minibatch/chunk             
            X_batch = X_train[i:i + minibatch_size]
            y_batch = y_train[i:i + minibatch_size]

            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = torch.from_numpy(X_batch).to(device), torch.from_numpy(y_batch).to(device).long()


            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(True):
                outputs = model(inputs.float())
                _, preds = torch.max(outputs, 1)
                
                loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
                loss.backward()
                optimizer.step()
            
            
            # statistics
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

            # print ("losses_iter", loss.item() * inputs.size(0))
            # print ("accuracy_iter", torch.sum(preds == labels.data).item() / float(inputs.size(0)))

            losses_iter.append(loss.item())
            accuracy_iter.append(torch.sum(preds == labels.data).item() / float(inputs.size(0)))
        
        
        train_loss = np.mean(losses_iter)
        train_acc = np.mean(accuracy_iter)

        losses_train.append(train_loss)
        accuracy_train.append(train_acc)


        print('Training Loss: {:.4f} Acc: {:.4f}'.format(train_loss, train_acc))
        
        #Validation part
        model.eval()  # Set model to training mode
        
        inputs, labels = torch.from_numpy(X_val).to(device), torch.from_numpy(y_val).to(device).long()
        
        optimizer.zero_grad()
        
        outputs = model(inputs.float())
        _, preds = torch.max(outputs, 1)

        loss = criterion(outputs, labels)
        
        val_loss = loss.item()
        scheduler.step(val_loss)
        
        val_acc = torch.sum(preds == labels.data).item() / float(inputs.size(0))
        
        losses_val.append(val_loss)
        accuracy_val.append(val_acc)
        
        
        print('Validation Loss: {:.4f} Acc: {:.4f}'.format(val_loss, val_acc))

#         deep copy the model
        if val_acc > best_acc:
            best_train_acc = train_acc
            best_val_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())
            path = "best_model.pt"
            torch.save(best_model_wts, path)

        print()


    time_elapsed = time.time() - since
    print('Training complete in {:.0f}s'.format(time_elapsed))
    print('Best Train Acc: {:4f}'.format(best_train_acc))
    print('Best Val Acc: {:4f}'.format(best_val_acc))
    
    
#     stats_columns = ['Layers', 'Epochs', 'BatchSize', 'LearningRate', 'Optimizer', 'Scheduler', 'TrainAcc', 'ValAcc']
    layers = [module for module in model.modules() if type(module) != nn.Sequential]
    write_results([layers, num_epochs, minibatch_size, learning_rate, optimizer.state_dict, scheduler.state_dict, best_train_acc, best_val_acc])

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.drop_layer = nn.Dropout(p=0.4)
        self.fc1 = nn.Linear(37, 800)
        self.fc2 = nn.Linear(800, 600)
        self.fc3 = nn.Linear(600, 300)
        self.fc4 = nn.Linear(300, 100)
        self.fc5 = nn.Linear(100, 50)
        self.fc6 = nn.Linear(50, 4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc2(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc3(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc4(x))
        x = self.drop_layer(x)
        x = F.relu(self.fc5(x))
        x = self.drop_layer(x)
        x = self.fc6(x)
        return x

n_epochs = 10
minibatch_size = 128
learning_rate = 0.001
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
model = Net()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# scheduler = lr_scheduler.CosineAnnealingLR(optimizer, X_train.shape[0], eta_min=learning_rate)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
final_model = train_model(Xtrain, ytrain, Xval, yval, model, criterion, optimizer, scheduler, minibatch_size, num_epochs=n_epochs)

Epoch 0/9
----------
Training Loss: 0.3261 Acc: 0.8816


RuntimeError: CUDA out of memory. Tried to allocate 268.00 MiB (GPU 0; 4.00 GiB total capacity; 2.82 GiB already allocated; 257.26 MiB free; 15.74 MiB cached)

In [12]:
predict(X_test, y_test, final_model)

Test data, Loss: 0.138, Accuracy: 0.9471


In [53]:
X_trainM.shape

Xtrain.shape

(1326141, 37)

In [44]:
torch.cuda.empty_cache()

with open('dataset_mustafa.pkl', 'rb') as f:  # Python 3: open(..., 'wb')
    X_trainM, X_valM, X_testM, y_trainM, y_valM, y_testM = pickle.load(f)
    
scalerM = StandardScaler()
scalerM.fit(X_trainM)

X_trainM = scalerM.transform(X_trainM)
X_valM = scalerM.transform(X_valM)
X_testM = scalerM.transform(X_testM)

In [48]:
for i in range(1000):
    torch.cuda.empty_cache()

predict(X_valM, y_valM, final_model)

Test data, Loss: 0.957, Accuracy: 0.7829


In [49]:
val_predM = forest.predict(X_valM)
test_predM = forest.predict(X_testM)

val_accM = accuracy_score(y_valM, val_predM)
test_accM = accuracy_score(y_testM, test_predM)

print ("Random Forest, Val Acc: {0:.3}, Test Acc: {1:.3}".format(val_accM, test_accM))

Random Forest, Val Acc: 0.443, Test Acc: 0.444


# SVM

In [ ]:
# svc = svm.SVC(kernel='rbf')
# svc.fit(X_train, y_train)

# svc = svc.fit(X_train, y_train)

# train_pred = svc.predict(X_train)
# val_pred = svc.predict(X_val)
# test_pred = svc.predict(X_test)


# train_acc = accuracy_score(y_train, train_pred)
# val_acc = accuracy_score(y_val, val_pred)
# test_acc = accuracy_score(y_test, test_pred)

# print ("SVM Train Acc: {0:.3}, Val Acc: {1:.3}, Test Acc: {2:.3}".format(train_acc, val_acc, test_acc))